In [4]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
from google.colab import drive
drive.mount('/content/drive')
results_models_path = "/content/drive/MyDrive/DeepLearning_2024/Project/Results/"

Mounted at /content/drive


In [7]:
#Reading data from Google Drive
#This dataset is private, so I cannot share
weather = pd.read_csv("/content/drive/MyDrive/DeepLearning_2024/Project/barcelona last10years.csv", on_bad_lines='skip')
weather = weather[['datetime', 'tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob' ,'preciptype', 'windspeed', 'winddir', 'cloudcover']]

In [8]:
# Load dataset
data = pd.read_csv("/content/drive/MyDrive/DeepLearning_2024/Project/barcelona last10years.csv", on_bad_lines='skip')
data = data[['datetime', 'tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob' ,'preciptype', 'windspeed', 'winddir', 'cloudcover']]
data = data.rename(columns={'temp': 'temperature', 'datetime': 'date'})
data['date'] = pd.to_datetime(data['date'], utc=True)
data.set_index('date')
data['preciptype'] =  data['preciptype'].fillna('None')
data['cloudcover'] =  data['cloudcover'].fillna(0)
#data = data.resample('D').mean()
#data = data[['Temperature (C)']]

In [9]:
class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super(LSTM, self).__init__()
    self.M = hidden_dim
    self.L = layer_dim

    self.rnn = nn.LSTM(
        input_size=input_dim,
        hidden_size=hidden_dim,
        num_layers=layer_dim,
        batch_first=True)
    #batch_first to have (batch_dim, seq_dim, feature_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, X):
    # initial hidden state and cell state
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
    c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    out, (hn, cn) = self.rnn(X, (h0.detach(), c0.detach()))

    # h(T) at the final time step
    out = self.fc(out[:, -1, :])
    return out

In [10]:
#One-hot-encoding precipitation type and month
data['month'] = data['date'].dt.month
weather_LSTM = data.copy(deep=True)
weather_LSTM = pd.get_dummies(data, columns = ['preciptype', 'month'])
weather_LSTM.columns

Index(['date', 'tempmax', 'tempmin', 'temperature', 'dew', 'humidity',
       'precip', 'precipprob', 'windspeed', 'winddir', 'cloudcover',
       'preciptype_None', 'preciptype_rain', 'preciptype_rain,snow', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [11]:
input_data = weather_LSTM.drop(['date'], axis=1)
targets = weather_LSTM['temperature'].values

T = 20                   #Number of timesteps to look while predicting (BY DEFAULT --> 20)
D = input_data.shape[1]   #Dimensionality of the input
N = len(input_data) - T
print(f'Dimensions are {T} × {D} × {N}')

#Train size: 80% of the total data size
train_size = int(len(input_data) * 0.80)

# Normalization of the inputs
scaler = StandardScaler()
scaler.fit(input_data[:train_size + T - 1])
input_data = scaler.transform(input_data)

# Preparing X_train and y_train
X_train = np.zeros((train_size, T, D))
y_train = np.zeros((train_size, 1))

for t in range(train_size):
  X_train[t, :, :] = input_data[t:t+T]
  y_train[t] = (targets[t+T])


# Preparing X_test and y_test
X_test = np.zeros((N - train_size, T, D))
y_test = np.zeros((N - train_size, 1))

for i in range(N - train_size):
  t = i + train_size
  X_test[i, :, :] = input_data[t:t+T]
  y_test[i] = (targets[t+T])

# Make inputs and targets
X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

Dimensions are 20 × 25 × 3775


In [14]:
model = LSTM(D, 512, 2, 1)
model.load_state_dict(torch.load(results_models_path + "model1.ckpt"))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DeepLearning_2024/Project/Results/model1.ckpt'

In [ ]:
#Checking one-step prediction performance of the model
test_target = y_test.cpu().detach().numpy()
test_predictions = []

for i in range(len(test_target)):
  input_ = X_test[i].reshape(1, T, D)
  p = model(input_)[0,0].item()

  # update the predictions list
  test_predictions.append(p)
